In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Nathan's path
%cd drive/MyDrive/work/S2/NLP/medal

#Someone else's path
#%cd /content/drive/MyDrive/Dauphine/M2/S2/NLP/medal

/content/drive/MyDrive/work/S2/NLP/medal


In [3]:
%pip install transformers fasttext -q

In [ ]:
#!bash run.sh

In [4]:
%run preprocess/mimic.py --save_dir ./data --mimic_dir ./data/mimic/

/usr/local/lib/python3.7/dist-packages/IPython/utils/py3compat.py:186: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(compiler(f.read(), fname, 'exec'), glob, loc)
100%|██████████| 260203/260203 [00:18<00:00, 14317.56it/s]


In [ ]:
!bash run_downstream.sh

configurations:
savedir: ./csv_logs
model: electra
task: mimic-mortality
test: False
data_dir: ./data/downstream
data_filename: mimic.csv
diag_to_idx_path: None
embs_path: None
pretrained_model: ./models/electra.pt
use_scheduler: True
lr: 2e-06
dropout: 0.1
epochs: 3
save_every: 1
eval_every: 10000
batchsize: 8
hidden_size: 512
rnn_layers: 3
da_layers: 1
ncpu: 4
    HADM_ID  ...  HOSPITAL_EXPIRE_FLAG
0  116532.0  ...                   1.0
1  116532.0  ...                   1.0
2  111458.0  ...                   1.0
3  116532.0  ...                   1.0
4  116532.0  ...                   1.0

[5 rows x 4 columns]
Data loaded
Dataset created
Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing 

In [14]:
import pandas as pd
import numpy as np
df = pd.read_csv('./data/test.csv')
df.head()

df['LABEL_NUM'] = df.HOSPITAL_EXPIRE_FLAG.astype(np.float32)

In [15]:
df

,HADM_ID,SUBJECT_ID,TEXT,HOSPITAL_EXPIRE_FLAG,LABEL_NUM
0,146431.0,31916,"respiratory failure , acute ( not ards / doc...",1.0,1.0
1,146431.0,31916,chief complaint : nausea and vomiting\n hp...,1.0,1.0
2,146431.0,31916,chief complaint : nausea and vomiting\n hp...,1.0,1.0
3,181664.0,29080,chief complaint : fatigue and dizziness\n ...,1.0,1.0
4,181664.0,29080,"gastrointestinal bleed , other ( gi bleed , ...",1.0,1.0
...,...,...,...,...,...
43728,199972.0,81561,"pneumonia , other\n assessment : \n cxr f...",0.0,0.0
43729,199994.0,23761,neuro : pt . sedated with ativan iv but arou...,0.0,0.0
43730,199994.0,23761,micu / sicu npn\n s / o : cardiac : r ' d...,0.0,0.0
43731,199994.0,23761,respiratory care : \npatient self extubated th...,0.0,0.0


In [13]:
df.HOSPITAL_EXPIRE_FLAG

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
43728    0.0
43729    0.0
43730    0.0
43731    0.0
43732    0.0
Name: HOSPITAL_EXPIRE_FLAG, Length: 43733, dtype: float64

# IGNORE : Test of non bash command

In [ ]:
import argparse
import os
import time
import pandas as pd

import torch
import torch.optim as optim
from torch import nn

from models.rnn import RNN
from models.lstm_sa import RNNAtt
from models.electra import Electra
from transformers import ElectraTokenizer
from utils import load_dataframes, load_model, train_loop
from models.tokenizer_and_dataset import \
    FastTextTokenizer, EmbeddingsDataset, HuggingfaceDataset

from torch.utils.tensorboard import SummaryWriter


ARGS = {
    "savedir":"./run_logs",
    "model":"electra",
    "data_dir":"./data",
    "data_filename":"medal.csv",
    "adam_path":"./toy_data/valid_adam.txt",
    "embs_path":"./data",
    "use_scheduler":True,
    "lr":2e-6,
    "clip":0,
    "dropout":0.1,
    "epochs":10,
    "accum_num":1,
    "save_every":1,
    "eval_every":200000,
    "batchsize":8,
    "hidden_size":512,
    "rnn_layers":3,
    "da_layers":1,
    }

EXPERIMENT_DIR = ARGS.get('savedir')
N_EPOCHS = ARGS.get('epochs')
BATCH_SIZE = ARGS.get('batchsize')
N_CPU_CORES = ARGS.get('ncpu')
MODEL_TYPE = ARGS.get('model')
USE_PRETRAIN = False

DEVICE = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")

# Load data

train, valid, test, label_to_ix = load_dataframes (
                                    data_dir=ARGS.get('data_dir'), 
                                    data_filename=ARGS.get('data_filename'),
                                    adam_path=ARGS.get('adam_path')
                                    )
print("Data loaded")

# Create tokenizer objects
if MODEL_TYPE == "electra":
    tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
else:
    # Create word index and load Fasttext embedding matrix
    tokenizer = FastTextTokenizer(verbose=True)
    tokenizer.build_word_index(train.TEXT, valid.TEXT, test.TEXT, list(label_to_ix.keys()))
    tokenizer.build_embedding_matrix(ARGS.get('embs_path'))

# Create torch Dataset objects
if MODEL_TYPE == "electra":
    train_data = HuggingfaceDataset(train, tokenizer=tokenizer, device=DEVICE)
    valid_data = HuggingfaceDataset(valid, tokenizer=tokenizer, device=DEVICE)
else:
    train_data = EmbeddingsDataset(train, tokenizer=tokenizer, device=DEVICE)
    valid_data = EmbeddingsDataset(valid, tokenizer=tokenizer, device=DEVICE)
print("Dataset created")

# Define network, loss function and optimizer
# if MODEL_TYPE == "rnn":   # reproduce from Xing Han
#     LSTM_PARAMS = dict(
#         batch_first=True,
#         num_layers=args.rnn_layers,
#         dropout=args.dropout,
#         bidirectional=True,
#         hidden_size=args.hidden_size
#     )
#     net = RNN(
#         embedding_dim=tokenizer.embedding_matrix.shape[1],
#         output_size=len(label_to_ix),
#         rnn_params=LSTM_PARAMS,
#         device=DEVICE
#     )
# elif MODEL_TYPE == "rnnsoft":
#     net = RNNAtt(
#         rnn_layers=args.rnn_layers,
#         da_layers=args.da_layers,
#         output_size=len(label_to_ix),
#         d_model=args.hidden_size,
#         device=DEVICE,
#         dropout_rate=args.dropout,
#     )
# elif MODEL_TYPE == "electra":
net = Electra(
        output_size=len(label_to_ix),
        device=DEVICE,
    )
print('model: {}'.format(net))


if torch.cuda.device_count() > 1:
    net.to(DEVICE)
    print("Using", torch.cuda.device_count(), "GPUs")
    net = nn.DataParallel(net)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), ARGS.get('lr'))
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=8) \
    if ARGS.get('use_scheduler') else None

# Create save directory
time_stamp = time.strftime("%m-%d-%H-%M", time.localtime())
save_dir = os.path.join(EXPERIMENT_DIR, time_stamp)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Save configs
model_desc_output = [": ".join([str(k), str(v)]) for k, v in ARGS.items()]
with open(os.path.join(save_dir, 'configs.txt'), 'w') as file:
    file.writelines("\n".join(model_desc_output))

# Set up tensorboard
writer = SummaryWriter(f"runs/{MODEL_TYPE}-{time_stamp}")

# Train network
net, logs = train_loop(
    net, MODEL_TYPE, optimizer, criterion, train_data, valid_data, save_dir=save_dir, n_epochs=N_EPOCHS, \
        batch_size=BATCH_SIZE, verbose=True, scheduler=scheduler, save_every=ARGS.get('save_every'), \
        eval_every=ARGS.get('eval_every'), clip=ARGS.get('clip'), writer=writer, accum_num=ARGS.get('accum_num'),
)

# Save Model
torch.save(net, os.path.join(save_dir, 'model.pt'))

# Save Logs
log_df = pd.DataFrame(logs)
log_df.to_csv(os.path.join(save_dir, 'logs.csv'))


Data loaded


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Dataset created
model: Electra(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_fe

  0%|          | 0/375000 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
  2%|▏         | 9001/375000 [23:47<16:10:29,  6.29it/s]